## <font color='Blue'>Introduction</font>
###  In this exercise, we run simulation in a 2D single layer two-phase model.
* The realistic formation heterogeneity is used for permeability map. 

## <font color='blue'>The objectives:</font>
1. Introduce custom <font color='red'>'Model'</font> class based on 
 * Class [DartsModel](darts/models/darts_model.py) with base model capabilities
 * Class [StructReservoir](darts/models/reservoirs/struct_reservoir.py) with structured reservoir
 * Class [Deadoil](darts/models/physics/dead_oil_python.py) for basic dead oil physics
 * Class [Deadoil-properties](darts/models/physics/do_properties_python.py) for dead oil properties, such as density, viscosity and relative permeability
2. Use run_python procedure to control run from the python script
3. Introduce wells and time-dependent well controls
4. Redefine physical properties and run simulation with custom-defined property.

## <font color='Blue'>Let's start!</font>


<img style="float: left;" src="slides/Slide33.JPG" width="60%">

In [ ]:
from darts.models.reservoirs.struct_reservoir import StructReservoir
from darts.models.physics.dead_oil_python import DeadOil
from darts.models.darts_model import DartsModel
from darts.engines import value_vector
from darts.tools.keyword_file_tools import load_single_keyword
import numpy as np

<img style="float: left;" src="slides/Slide34.JPG" width="60%">

#### Brief Introduction of model inheritance
* Here create the <font color='red'>'Model' </font>  class, which inherits from <font color='red'>DartsModel</font> (the base class).
* It keeps all the functionalities of <font color='red'>DartsModel</font> and can also be extended to add more functionalities.
* If a function is redefined in subclass, the function in base class with identical name will be overridden.

In [ ]:
class Model(DartsModel):
    def __init__(self):
        # call base class constructor
        super().__init__()

        # measure time spend on reading/initialization
        self.timer.node["initialization"].start()

        # create reservoir
        self.nx = 60
        self.ny = 60
        self.nz = 1
        
        self.dx = 8
        self.dy = 8
        self.dz = 4

#         self.permx = np.ones(self.nx*self.ny)*1000
#         self.actnum = np.ones(self.nx*self.ny)
        
        self.permx = load_single_keyword('Egg/data.in','PERMX')
        self.actnum = load_single_keyword('Egg/data.in','ACTNUM')       

        self.permy = self.permx
        self.permz = 0.1 * self.permx
        self.poro = 0.2
        self.depth = 4000

        # run discretization
        self.reservoir = StructReservoir(self.timer, nx=self.nx, ny=self.ny, nz=self.nz, dx=self.dx, dy=self.dy,
                                         dz=self.dz, permx=self.permx, permy=self.permy, permz=self.permz,
                                         poro=self.poro, depth=self.depth,actnum=self.actnum)

        # add two wells
        well_diam = 0.2
        well_rad = well_diam/2


            
#         self.reservoir.add_well("INJ1")
#         self.reservoir.add_perforation(self.reservoir.wells[-1], 5, 57, 1, well_radius=well_rad, multi_segment=False)
#         self.reservoir.inj_wells = [self.reservoir.wells[-1]]        

#         self.reservoir.add_well("PRD1")
#         self.reservoir.add_perforation(self.reservoir.wells[-1], 43, 18, 1, well_radius=well_rad, multi_segment=False)
#         self.reservoir.prod_wells = [self.reservoir.wells[-1]]

        inj_loc = [(5, 57), (29, 53), (4, 35), (27, 29), (49, 35), (10, 9), (32, 3), (57, 7)]
        prod_loc = [(16, 43), (35, 40), (23, 16), (43, 18)]
        for i in range(len(inj_loc)):
            self.reservoir.add_well("INJ"+str(i+1))
            self.reservoir.add_perforation(self.reservoir.wells[-1], int(inj_loc[i][0]), int(inj_loc[i][1]), 1, well_radius=well_rad, multi_segment=False)
            self.reservoir.inj_wells = [self.reservoir.wells[-1]]
        for i in range(len(prod_loc)):
            self.reservoir.add_well("PRD"+str(i+1))
            self.reservoir.add_perforation(self.reservoir.wells[-1], int(prod_loc[i][0]), int(prod_loc[i][1]), 1, well_radius=well_rad, multi_segment=False)
            self.reservoir.prod_wells = [self.reservoir.wells[-1]]

        # create physics
        self.physics = DeadOil(timer=self.timer, physics_filename='Egg/physics.in',
                               n_points=201, min_p=300, max_p=500, min_z=1e-8)

        self.params.first_ts = 1
        self.params.mult_ts = 2
        self.params.max_ts = 50
        self.params.tolerance_newton = 1e-2
        self.params.tolerance_linear = 1e-3

        self.inj = [1]

        self.runtime = 500

        self.timer.node["initialization"].stop()

    def set_initial_conditions(self):
        self.physics.set_uniform_initial_conditions(self.reservoir.mesh, uniform_pressure=400,
                                                    uniform_composition=[2e-2])

    def set_boundary_conditions(self):
        for i, w in enumerate(self.reservoir.wells):
            if w.name[:3] == "INJ":
                w.control = self.physics.new_bhp_water_inj(410, self.inj)
                # w.constraint = self.physics.new_rate_water_prod(210)
            else:
                w.control = self.physics.new_bhp_prod(390)

<img style="float: left;" src="slides/Slide35.JPG" width="60%">

In [ ]:
m = Model()
m.init()
m.run_python()
m.print_timers()

<img style="float: left;" src="slides/Slide36.JPG" width="60%">

In [ ]:
%matplotlib inline
import pandas as pd
time_data = pd.DataFrame.from_dict(m.physics.engine.time_data)
# wirte timedata to output file
time_data.to_pickle("darts_time_data.pkl")
# write timedata to excel file
writer = pd.ExcelWriter('time_data.xlsx')
time_data.to_excel(writer, 'Sheet1')
writer.save()

<img style="float: left;" src="slides/Slide37.JPG" width="60%">

### Next we process the active grid, prepare plotting and plot pressure

In [ ]:
import matplotlib.pyplot as plt
from darts.models.physics.do_operator_python import *

# process active grid
X = np.array(m.physics.engine.X, copy=False)
nb = m.nx * m.ny
nb = np.count_nonzero(m.actnum)

p = -np.ones(m.nx*m.ny)
z = -np.ones(m.nx*m.ny)
s = -np.ones(m.nx*m.ny)

p[act>0] = X[0:2*nb:2]
z[act>0] = X[1:2*nb:2]

plt.figure(num=2, figsize=(12,4))
plt.subplot(121)
plt.pcolor(p.reshape(m.ny, m.nx),cmap='jet')
plt.clim([390,410])
plt.colorbar()


<img style="float: left;" src="slides/Slide38.JPG" width="60%">

<img style="float: left;" src="slides/Slide40.JPG" width="60%">

<img style="float: left;" src="slides/Slide41.JPG" width="60%">

<img style="float: left;" src="slides/Slide42.JPG" width="60%">

<img style="float: left;" src="slides/Slide43.JPG" width="60%">